# 0. Install and Import Dependencies

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 1. Load Model

In [3]:
module = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
model = module.signatures['serving_default']

# 2. Make Detections

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)

In [5]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [6]:
EDGES = {
    (0, 1): (0,0,255),
    (0, 2): (0,255,0),
    (1, 3): (0,0,255),
    (2, 4): (0,255,0),
    (0, 5): (0,0,255),
    (0, 6): (0,255,0),
    (5, 7): (0,0,255),
    (7, 9): (0,0,255),
    (6, 8): (0,255,0),
    (8, 10): (238, 99, 99), # 右手臂
    (5, 6): (70,140,105), # 身體上槓
    (5, 11): (0,0,255),
    (6, 12): (0,255,0),
    (11, 12): (70,140,105), # 身體下槓
    (11, 13): (0,0,255),
    (13, 15): (0,0,255),
    (12, 14): (0,255,0),
    (14, 16): (0,255,0)
}

In [7]:
def movenet(input_image):
    """Runs detection on an input image.

    """
    model = module.signatures['serving_default']
    input_image = tf.image.resize_with_pad(tf.expand_dims(input_image, axis=0), 384,640)
    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    return keypoints_with_scores

In [8]:
cap = cv2.VideoCapture(1)
counter = 0
import time
while cap.isOpened():

    ret, frame = cap.read()
    start_time = time.time()
    # # Resize imageq
    img = frame.copy()

    keypoints_with_scores = movenet(img)

    # Render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    counter += 1  # 计算帧数
    cv2.putText(frame, "FPS {0}".format(float('%.1f' % (1 / (time.time() - start_time)))), (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
    cv2.imshow('Movenet Multipose', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt

